In [43]:
# Importing required libraries and packages
import json 
import numpy as np 
import random
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder



Data Preprocessing

In [44]:
# Load and preprocess the data
with open('intents.json') as file:
  data = json.load(file)

# Hold all the training data    
training_sentences = []
#Hold all target labels that cprrespond to each training data
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for question in intent['questions']:
        training_sentences.append(question)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [45]:
# Define a function to get the best match for a given question
def get_best_match(question):
    # Set a threshold for the minimum score
    threshold = 70
    
    # Initialize variables to store the best match and its score
    best_match = None
    best_score = 0
    
    # Loop through all the questions in the data
    for item in data:
        # Compute the score for the current question
        score = fuzz.token_set_ratio(question, item['question'])
        
        # If the score is above the threshold and better than the previous best score, update the best match
        if score > threshold and score > best_score:
            best_match = item['responses']
            best_score = score
    
    # If there is a best match, return it. Otherwise, return a default message.
    if best_match is not None:
        return best_match
    else:
        return "I'm sorry, I don't understand your question."

In [46]:
# To convert target label into a model understandable form
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [47]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"
# To vectorize the whole text data 
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# to make all the training text sequence into the same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

Model Design and Training

In [48]:
# Define the model architecture
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

model.add(SimpleRNN(16))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compiling the model
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

# Applying earlystopping to avoid model overfitting
earlystop_callback = EarlyStopping(monitor='loss', patience=5)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 16)            16000     
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 72)                1224      
                                                                 
Total params: 18,296
Trainable params: 18,296
Non-trainable params: 0
_________________________________________________________________


In [49]:
epochs = 150
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, callbacks=[earlystop_callback])

Epoch 1/150
8/8 [==============================] - 3s 145ms/step - loss: 4.2802 - accuracy: 0.0000e+00
Epoch 2/150
8/8 [==============================] - 1s 170ms/step - loss: 4.2721 - accuracy: 0.0088
Epoch 3/150
8/8 [==============================] - 0s 66ms/step - loss: 4.2628 - accuracy: 0.0354
Epoch 4/150
8/8 [==============================] - 1s 86ms/step - loss: 4.2540 - accuracy: 0.0398
Epoch 5/150
8/8 [==============================] - 0s 65ms/step - loss: 4.2407 - accuracy: 0.0708
Epoch 6/150
8/8 [==============================] - 0s 45ms/step - loss: 4.2248 - accuracy: 0.0841
Epoch 7/150
8/8 [==============================] - 0s 46ms/step - loss: 4.2041 - accuracy: 0.0885
Epoch 8/150
8/8 [==============================] - 0s 42ms/step - loss: 4.1767 - accuracy: 0.0841
Epoch 9/150
8/8 [==============================] - 0s 44ms/step - loss: 4.1465 - accuracy: 0.1062
Epoch 10/150
8/8 [==============================] - 0s 44ms/step - loss: 4.1080 - accuracy: 0.0664
Epoch 11/150


In [50]:
# to save the trained model
model.save("chat_model")

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Model Testing and Evaluation

In [51]:
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit":
          print("AruBot: Goodbye!")
          break
        
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        for i in data['intents']:
          if i['tag'] == tag:
            print("AruBot:", np.random.choice(i['responses']))

print("Start messaging with the bot (type quit to stop)!")
chat()


Start messaging with the bot (type quit to stop)!
User: hello
1/1 [==============================] - 0s 178ms/step
AruBot: Hi
User: quit
AruBot: Goodbye!
